# Import potrebných knižníc

In [19]:
import re
import requests
from bs4 import BeautifulSoup
import os

# Stiahnutie snímkov z archívu

Sťahovanie z archívu je vytvorené pomocou web scrapingu kde po poskladaní URL odkazujúcej na deň z ktorého chceme stiahnuť snímky nočnej oblohy.

Zo stránky sú stiahnuté všetky súbory s koncovkou .gif (snímky sú len vo formáte .gif) 

Snímky sú následne uložené podľa roku, mesiaca a dňa.

In [20]:
#úvodné nastvenia premenných teda stanice, dní, mesiaca a filtra
location = "colombia"
start_day = 7
end_day = 30
month = "Oct"
year = "2021"
filt = "6300"

In [ ]:
#iterácia od úvodného po konečný deň
while start_day <= end_day:
    #http://sirius.bu.edu/data/?location=arecibo&year=2020&filt=6300&month=Jan&day=1
    #link vedúci ku dňu s dátami
    site = 'http://sirius.bu.edu/data/?location=' + location + '&year=' + year + '&filt=' + filt + '&month=' + month + '&day=' + str(start_day)
    response = requests.get(site)
    #pomodou knižnice BeautifulSoup parsujeme stránku
    soup = BeautifulSoup(response.text, 'html.parser')
    #nájdeme všetky položky s tagom img (obrázky)
    image_tags = soup.find_all('img')
    #zoznam všekých url obrázkov
    urls = [img['src'] for img in image_tags]
    #príprava priečinkov
    if not os.path.exists(location + year + "/" + month + "/" + str(start_day)):
        os.makedirs(location + year + "/" + month + "/" + str(start_day))
    #iterácia cez všetky url
    for url in urls:
        #získanie každého súboru vo formáte gif z url obrázkov
        filename = re.search(r'/([\w_-]+[.](gif))$', url)
        if not filename:
                continue
        #ukladanie jednotlivých obrázkov
        with open(location + year + "/" + month + "/" + str(start_day) + "/" + filename.group(1), 'wb') as f:
            if 'http' not in url:
                url = '{}{}'.format(site, url)
                url = url.split("data")
                url = url[0] + "data/data" + url[2]
            response = requests.get(url)
            f.write(response.content)
    start_day = start_day + 1